### Importations et configuration du chemin racine du projet

### Veuillez exécuter les cellules dans l’ordre

In [165]:
%run ./notebook_2018_extract.ipynb
import pandas as pd  # pour la manipulation de DataFrames
import os             # pour gérer les chemins et interactions système
import yaml           # pour lire et écrire des fichiers YAML
from pathlib import Path  # pour manipuler les chemins de fichiers de manière portable
from typing import Dict   # pour typer les dictionnaires dans les fonctions
import numpy as np    # pour les opérations numériques avancées

# Définir le chemin racine du projet
# Ici on prend le dossier parent du répertoire courant
PROJECT_ROOT = Path.cwd().parent

# Afficher le chemin racine pour vérification
PROJECT_ROOT


PosixPath('/home/kaouter/Bureau/projects/briefs_simplon/brief2/Brief-2-ETL-de-donn-es-footballistiques-Wickets-Sprinters')

### Fonctions utilisées pour la transformation des données

In [166]:


from pyparsing import col


def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Objectif :
        Charger les paramètres de configuration depuis un fichier YAML.
    Paramètres :
        config_filename (str) : Chemin relatif ou absolu du fichier YAML.
    Retour :
        dict : Paramètres de configuration.
    """
    config_path = Path(config_filename)

    # Résoudre le chemin relatif depuis la racine du projet
    if not config_path.is_absolute():
        project_root = Path(__file__).resolve().parents[1]
        config_path = project_root / config_path

    if not config_path.exists():
        raise FileNotFoundError(f"Fichier de configuration introuvable : {config_path}")

    # Charger le fichier YAML
    with open(config_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config

def fct_capitalize_string_columns(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """
    Objectif :
        Supprimer les espaces superflus et mettre la première lettre en majuscule,
        le reste en minuscules, uniquement pour les valeurs de type str.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        cols (list, optionnel) : Liste des colonnes à nettoyer.
    
    Retour :
        pd.DataFrame : DataFrame avec les colonnes de chaînes nettoyées.
    """
    # Vérifier que des colonnes ont été fournies
    if not cols:
        print("[INFO] Aucune colonne spécifiée pour capitalize_string_columns. Le DataFrame reste inchangé.")
        return df
    
    # Traiter chaque colonne spécifiée
    for col in cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype("string")
                .str.strip()
                .str.capitalize()
            )
    return df



def fct_upper_string_columns(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """
    Objectif :
        Supprimer les espaces superflus et mettre toutes les lettres en majuscules,
        uniquement pour les valeurs de type str.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        cols (list, optionnel) : Liste des colonnes à nettoyer. 
    Retour :
        pd.DataFrame : DataFrame avec les colonnes de chaînes nettoyées.
    """
    # Vérifier que des colonnes ont été fournies
    if not cols:
        print("[INFO] Aucune colonne spécifiée pour clean_string_columns_upper. Le DataFrame reste inchangé.")
        return df
    
    # Traiter chaque colonne spécifiée
    for col in cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype("string")
                .str.strip()
                .str.upper()
            )
    return df


def fct_lower_string_columns(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """
    Objectif :
        Supprimer les espaces superflus et mettre toutes les lettres en minuscules,
        uniquement pour les valeurs de type str.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        cols (list, optionnel) : Liste des colonnes à nettoyer. 
    Retour :
        pd.DataFrame : DataFrame avec les colonnes de chaînes nettoyées.
    """
    # Vérifier que des colonnes ont été fournies
    if not cols:
        print("[INFO] Aucune colonne spécifiée pour clean_string_columns_lower. Le DataFrame reste inchangé.")
        return df
    
    # Traiter chaque colonne spécifiée
    for col in cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype("string")
                .str.strip()
                .str.lower()
            )
    return df



def fct_iso_to_yyyymmddhhmmss(df: pd.DataFrame, col: str, new_col: str = None) -> pd.DataFrame:
    """
    Convertir une colonne ISO 8601 en une seule colonne YYYYMMDDhhmmss.
    Les valeurs manquantes ou invalides sont remplacées par :
        année=9999, mois=99, jour=99, heure=99, minute=99, seconde=99
    """
    if new_col is None:
        new_col = col

    def safe_convert(x):
        try:
            dt = pd.to_datetime(x, utc=True)
            year = f"{dt.year:04d}"
            month = f"{dt.month:02d}"
            day = f"{dt.day:02d}"
            hour = f"{dt.hour:02d}"
            minute = f"{dt.minute:02d}"
            second = f"{dt.second:02d}"
        except Exception:
            year = "9999"
            month = day = hour = minute = second = "99"

        return f"{year}{month}{day}{hour}{minute}{second}"

    df[new_col] = df[col].apply(safe_convert)
    return df



def fct_extract_edition(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Extraire l'année d'une colonne de date ISO 8601.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée
        col (str) : colonne ISO à extraire l'année
    
    Retour :
        pd.DataFrame : DataFrame avec nouvelle colonne année
    """
    df['edition'] = df[col].str[:4].astype(int)
    return df

def fct_generate_unique_stage(df: pd.DataFrame, col_stage:str , col_round:str, col_group : str) -> pd.DataFrame:
    """
    Générer une colonne unique 'stage' en combinant les colonnes 'round_id' et 'group_id'.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée
        col_stage (str) : nom de la colonne stage à créer
        col_round (str) : nom de la colonne round_id
        col_group (str) : nom de la colonne group_id
    
    Retour :
        pd.DataFrame : DataFrame avec nouvelle colonne stage
    """
    # générer une colonne stage_name
    conditions = [
        df['stage'].str.lower().str.strip() == 'group',
        df['stage'].str.lower().str.strip() == 'knockout'
    ]
    choices = [
    df['group_id'],
    df['round_id']
    ]
    df['stage_name'] = np.select(conditions, choices,  default='notdefined')
    return df


def fct_final_columns_to_keep (df : pd.DataFrame, columns_to_keep_original_list : list, columns_to_keep_final_list : list) -> pd.DataFrame:
    """
    Garder uniquement les colonnes spécifiées dans le DataFrame.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée
        columns_to_keep_original_list (list) : liste des noms de colonnes originales à garder
        columns_to_keep_final_list (list) : liste des nouveaux noms de colonnes à garder
    
    Retour :
        pd.DataFrame : DataFrame avec uniquement les colonnes spécifiées
    """
    # Créer un dictionnaire de mappage des anciennes aux nouvelles colonnes
    column_mapping = dict(zip(columns_to_keep_original_list, columns_to_keep_final_list))
    
    # Filtrer les colonnes à garder
    df_filtered = df[columns_to_keep_original_list].copy()
    
    # Renommer les colonnes selon le mapping
    df_filtered.rename(columns=column_mapping, inplace=True)
    
    return df_filtered

def fct_harmonize_column_values(df:pd.DataFrame, col:str , mapping_dict:Dict[str, str]) -> pd.DataFrame:
    """
    Harmoniser les valeurs d'une colonne en utilisant un dictionnaire de mappage.
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée
        col (str) : nom de la colonne à harmoniser
        mapping_dict (Dict[str, str]) : dictionnaire de mappage des valeurs
    Retour :
        pd.DataFrame : DataFrame avec la colonne harmonisée
    """
    if col not in df.columns:
        print(f"La colonne '{col}' n'existe pas dans le DataFrame.")
        return df
    
    df[col] = df[col].apply(lambda x: mapping_dict.get(x, x) if pd.notnull(x) else x)
    
    return df  

def fct_fillna_and_convert_types(
    df: pd.DataFrame,
) -> pd.DataFrame:
    """
    Remplit les valeurs manquantes dans une colonne avec une valeur spécifiée   
    """
    for col in df.columns:
        if df[col].dtype == 'string' or df[col].dtype == 'object':
            df[col] = df[col].fillna('notdefined').astype("string")
        elif df[col].dtype.name == 'Int64':
            df[col] = df[col].fillna(-999).astype("Int64")
        elif df[col].dtype == 'float64':
            df[col] = df[col].fillna(-999.0).astype("Float64")
        elif df[col].dtype == 'datetime64[ns, UTC]':
            df[col] = df[col].fillna(pd.Timestamp('9999-12-31 23:59:59', tz='UTC'))
        else:
            # Pour tous les autres types (bool, category, etc.)
            if df[col].isna().any():
                df[col] = df[col].fillna('notdefined')
    return df

In [167]:
#Récupération de la configuration
config_path = os.path.join(Path.cwd().parent, 'config.yaml')
config = fct_load_config(config_path)

# Transform

### Information sur les données 

In [168]:
df_infos = df_matches.copy()
df_infos['date_utc'] = pd.to_datetime(df_infos['date'], errors='coerce', utc=False)
# Calculer date début et date fin
date_debut = df_infos['date_utc'].min()
date_fin = df_infos['date_utc'].max()

print(f"Date début des matchs : {date_debut}")
print(f"Date fin des matchs : {date_fin}")

Date début des matchs : 2018-06-14 18:00:00+03:00
Date fin des matchs : 2018-07-15 18:00:00+03:00


/tmp/ipykernel_81657/3466716.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_infos['date_utc'] = pd.to_datetime(df_infos['date'], errors='coerce', utc=False)


### Stadiums

In [169]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des stades : 
print(f"\nNombre des stades : {df_stadiums['id'].nunique()}")


# verification des doublons dans df_stadiums

# df_stadiums.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_stadiums.duplicated(subset = ['id']).sum()
print(f"\nNombre des 'id' des stadiums dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_stadiums:\n{df_stadiums.isnull().sum()}")

#types des colonnes
print(f"\nTypes des colonnes dans df_stadiums:\n{df_stadiums.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------
# Filtrer les colonnes nécessaires
columns_to_keep = ['id', 'name', 'city']
df_stadiums_transformed = df_stadiums[columns_to_keep].copy()

#traitement des valeurs nulles
df_stadiums_transformed = fct_fillna_and_convert_types(df_stadiums_transformed)
#supprimer '.' à la fin des noms des villes
df_stadiums_transformed['city'] = df_stadiums_transformed['city'].str.replace(r'\.$', '', regex=True)

# Mettre en le premier caractère en Majuscule pour les colonnes name et city & convertir en type string
df_stadiums_transformed = fct_capitalize_string_columns(df_stadiums_transformed, ['name', 'city'])

print(f'\ndf_stadiums_transformed.dtypes:\n{df_stadiums_transformed.dtypes}')
df_stadiums_transformed.head()


Nombre des stades : 12

Nombre des 'id' des stadiums dupliqués: 0

Valeurs nulles dans df_stadiums:
id       0
name     0
city     0
lat      0
lng      0
image    0
dtype: int64

Types des colonnes dans df_stadiums:
id         int64
name      object
city      object
lat      float64
lng      float64
image     object
dtype: object

df_stadiums_transformed.dtypes:
id               int64
name    string[python]
city    string[python]
dtype: object


,id,name,city
0,1,Luzhniki stadium,Moscow
1,2,Otkrytiye arena,Moscow
2,3,Krestovsky stadium,Saint petersburg
3,4,Kaliningrad stadium,Kaliningrad
4,5,Kazan arena,Kazan


### tvchannels

In [170]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------
# verification des doublons dans df_channels

# Nombres des chaines télévisées : 
print(f"\nNombre des chaines télévisées : {df_tvchannels['id'].nunique()}")


# df_tvchannels.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_tvchannels.duplicated(subset = ['id']).sum()
print(f"\nNombre des 'id' des channels dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_tvchannels:\n{df_tvchannels.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_tvchannels:\n{df_tvchannels.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------
# Filtrer les colonnes nécessaires
columns_to_keep = ['id', 'name', 'country', 'lang']
df_tvchannels_transformed = df_tvchannels[columns_to_keep].copy()

#traitement des valeurs nulles
df_tvchannels_transformed = fct_fillna_and_convert_types(df_tvchannels_transformed)
# Mettre en le premier caractère en Majuscule pour la colonne country & convertir en type string
df_tvchannels_transformed = fct_capitalize_string_columns(df_tvchannels_transformed, ['country'])
# Eclater les listes dans la colonne 'lang' en plusieurs lignes
# df_tvchannels_transformed = df_tvchannels_transformed.explode('lang').reset_index(drop=True)

# Mettre en Majuscule pour les colonnes name et lang & convertir en type string
df_tvchannels_transformed = fct_upper_string_columns(df_tvchannels_transformed, ['name', 'lang'])



print(f'\ndf_tvchannels_transformed.dtypes:\n{df_tvchannels_transformed.dtypes}')
df_tvchannels_transformed.head()





Nombre des chaines télévisées : 22

Nombre des 'id' des channels dupliqués: 0

Valeurs nulles dans df_tvchannels:
id         0
name       0
icon       0
country    0
iso2       0
lang       0
dtype: int64

Types des colonnes dans df_tvchannels:
id          int64
name       object
icon       object
country    object
iso2       object
lang       object
dtype: object

df_tvchannels_transformed.dtypes:
id                  int64
name       string[python]
country    string[python]
lang       string[python]
dtype: object


,id,name,country,lang
0,1,DR1,Denmark,['DA']
1,2,TV2,Denmark,['DA']
2,3,BBC UK,Uk,['EN']
3,4,ITV UK,Uk,['EN']
4,5,ITV4 UK,Uk,['EN']


### teams

In [171]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des équipes : 
print(f"\nNombre des équipes : {df_teams['id'].nunique()}")

# verification des doublons dans df_teams
# df_teams.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_teams.duplicated(subset = ['id']).sum()
print(f"\nNombre des 'id' des teams dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_teams:\n{df_teams.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_teams:\n{df_teams.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Filtrer les colonnes nécessaires
columns_to_keep = ['id', 'name']
df_teams_transformed = df_teams[columns_to_keep].copy()

#traitement des valeurs nulles
df_teams_transformed = fct_fillna_and_convert_types(df_teams_transformed)

# Mettre en le premier caractère en Majuscule pour la colonne name & convertir en type string
df_teams_transformed = fct_capitalize_string_columns(df_teams_transformed, ['name'])

print(f'\ndf_teams_transformed.dtypes:\n{df_teams_transformed.dtypes}')
df_teams_transformed.head(48)


Nombre des équipes : 32

Nombre des 'id' des teams dupliqués: 0

Valeurs nulles dans df_teams:
id             0
name           0
fifaCode       0
iso2           0
flag           0
emoji          0
emojiString    0
dtype: int64

Types des colonnes dans df_teams:
id              int64
name           object
fifaCode       object
iso2           object
flag           object
emoji          object
emojiString    object
dtype: object

df_teams_transformed.dtypes:
id               int64
name    string[python]
dtype: object


,id,name
0,1,Russia
1,2,Saudi arabia
2,3,Egypt
3,4,Uruguay
4,5,Portugal
5,6,Spain
6,7,Morocco
7,8,Iran
8,9,France
9,10,Australia


### groups

In [172]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des groupes : 
print(f"\nNombre des groupes : {df_groups['group_id'].nunique()}")

# verification des doublons dans df_groups

# df_groups.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_groups.duplicated(subset = ['group_id']).sum()
print(f"\nNombre des 'group_id' des groups dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_groups:\n{df_groups.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_groups:\n{df_groups.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Mettre en le premier caractère en Majuscule pour la colonne group_name & convertir en type string
df_groups_transformed = fct_upper_string_columns(df_groups, ['group_name'])

#traitement des valeurs nulles
df_groups_transformed= fct_fillna_and_convert_types(df_groups_transformed)


# Mettre en le premier caractère en Muniscule pour la colonne group_id & convertir en type string
df_groups_transformed = fct_lower_string_columns(df_groups_transformed, ['group_id'])

print(f'\ndf_groups_transformed.dtypes:\n{df_groups_transformed.dtypes}')
df_groups_transformed.head(10)



Nombre des groupes : 8

Nombre des 'group_id' des groups dupliqués: 0

Valeurs nulles dans df_groups:
group_id            0
group_name          0
winner_team_id      0
runnerup_team_id    0
dtype: int64

Types des colonnes dans df_groups:
group_id            object
group_name          object
winner_team_id       int64
runnerup_team_id     int64
dtype: object

df_groups_transformed.dtypes:
group_id            string[python]
group_name          string[python]
winner_team_id               int64
runnerup_team_id             int64
dtype: object


,group_id,group_name,winner_team_id,runnerup_team_id
0,a,GROUP A,4,1
1,b,GROUP B,6,5
2,c,GROUP C,9,12
3,d,GROUP D,15,13
4,e,GROUP E,17,18
5,f,GROUP F,23,22
6,g,GROUP G,25,28
7,h,GROUP H,31,32


### rounds

In [173]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des phases : 
print(f"\nNombre des phases : {df_rounds['round_id'].nunique()}")

# verification des doublons dans df_rounds

# df_rounds.groupby('round_id').size().loc[lambda x: x > 1]

number_duplicates = df_rounds.duplicated(subset = ['round_id']).sum()
print(f"\nNombre des 'round_id' des rounds dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_rounds:\n{df_rounds.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_rounds:\n{df_rounds.dtypes}")
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Mettre en le premier caractère en Minuscule pour la colonne round_id & convertir en type string
df_rounds_transformed = fct_lower_string_columns(df_rounds, ['round_id'])

#traitement des valeurs nulles
df_rounds_transformed = fct_fillna_and_convert_types(df_rounds_transformed)

# Mettre en le premier caractère en Majuscule pour la colonne round_name & convertir en type string
df_rounds_transformed = fct_capitalize_string_columns(df_rounds_transformed, ['round_name'])

print(f'\ndf_rounds_transformed.dtypes:\n{df_rounds_transformed.dtypes}')
df_rounds_transformed.head()



Nombre des phases : 5

Nombre des 'round_id' des rounds dupliqués: 0

Valeurs nulles dans df_rounds:
round_id      0
round_name    0
dtype: int64

Types des colonnes dans df_rounds:
round_id      object
round_name    object
dtype: object

df_rounds_transformed.dtypes:
round_id      string[python]
round_name    string[python]
dtype: object


,round_id,round_name
0,round_16,Round of 16
1,round_8,Quarter-finals
2,round_4,Semi-finals
3,round_2_loser,Third place play-off
4,round_2,Final


### matches

In [174]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des matches : 
print(f"\nNombre des matches : {df_matches['match_id'].nunique()}")

# Valeur stage : 
print(f"\nstage : {df_matches['stage'].unique().tolist()}")

# Valeur type : 
print(f"\ntype : {df_matches['type'].unique().tolist()}")

# ids des groups : 
print(f"\ngroup_id : {df_matches['group_id'].unique().tolist()}")

# ids des phases : 
print(f"\nround_id : {df_matches['round_id'].unique().tolist()}")

# verification des doublons dans df_matches

# df_matches.groupby('match_id').size().loc[lambda x: x > 1]

number_duplicates = df_matches.duplicated(subset = ['match_id']).sum()
print(f"\nNombre des 'match_id' des matches dupliqués: {number_duplicates}")

# valeurs nulles 
print(f"\nValeurs nulles dans df_matches:\n{df_matches.isnull().sum()}")  
# types des colonnes
print(f"\nTypes des colonnes dans df_matches:\n{df_matches.dtypes}")
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Mettre en le premier caractère en Minuscule pour la colonne group_id et round_id & convertir en type string
df_matches_transformed = fct_lower_string_columns(df_matches, ['group_id','round_id'])

df_matches_transformed = fct_fillna_and_convert_types(df_matches_transformed)

# Mettre en le premier caractère en Majuscule pour la colonne stage et type & convertir en type string
df_matches_transformed = fct_capitalize_string_columns(df_matches_transformed, ['stage','type'])

# Remplir les valeurs manquantes dans 'round_id' et 'group_id' avec 'notdefined'
df_matches_transformed['round_id'] = df_matches_transformed['round_id'].fillna('notdefined')
df_matches_transformed['group_id'] = df_matches_transformed['group_id'].fillna('notdefined')

# Générer une colonne unique 'stage' en combinant les colonnes 'round_id' et 'group_id'.
df_matches_transformed = fct_generate_unique_stage(df_matches_transformed, 'stage', 'round_id', 'group_id')
print(f"\nvaleur unique de stage_name : {df_matches_transformed['stage_name'].unique()}")

# Extraire l'année de la colonne date pour créer la colonne edition
df_matches_transformed['edition'] = fct_extract_edition(df_matches_transformed, 'date')['edition']

# Convertir la colonne date ISO en format YYYYMMDDhhmmss
df_matches_transformed = fct_iso_to_yyyymmddhhmmss(df_matches_transformed, 'date', 'formatted_date')

# # Eclater les listes dans la colonne 'channels' en plusieurs lignes
# df_matches_transformed = df_matches_transformed.explode('channels').reset_index(drop=True)

#Génerer une colonne unique 'stage' en combinant les colonnes 'round_id' et 'group_id'.
df_matches_transformed = fct_generate_unique_stage(df_matches_transformed, 'stage', 'round_id', 'group_id')

# Harmoniser la colonne stage_name en fonction des valeurs de stage
stage_mapping = config['stage_mapping_2018']
df_matches_transformed = fct_harmonize_column_values(df_matches_transformed, 'stage_name', stage_mapping)
print(f'\ndf_matches_transformed.dtypes:\n{df_matches_transformed.dtypes}')

df_matches_transformed.head()


Nombre des matches : 64

stage : ['group', 'knockout']

type : ['group', 'qualified', 'winner', 'loser']

group_id : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', None]

round_id : [None, 'round_16', 'round_8', 'round_4', 'round_2_loser', 'round_2']

Nombre des 'match_id' des matches dupliqués: 0

Valeurs nulles dans df_matches:
match_id         0
type             0
stage            0
group_id        16
round_id        48
date             0
stadium_id       0
home_team_id     0
away_team_id     0
home_result      0
away_result      0
home_penalty    60
away_penalty    60
winner          48
finished         0
matchday         0
channels         0
dtype: int64

Types des colonnes dans df_matches:
match_id          int64
type             object
stage            object
group_id         object
round_id         object
date             object
stadium_id        int64
home_team_id      int64
away_team_id      int64
home_result       int64
away_result       int64
home_penalty    float64
away_penalty

,match_id,type,stage,group_id,round_id,date,stadium_id,home_team_id,away_team_id,home_result,away_result,home_penalty,away_penalty,winner,finished,matchday,channels,stage_name,edition,formatted_date
0,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,0,-999.0,-999.0,-999.0,True,1,"[4, 6, 13, 17, 20, 22]",group_a,2018,20180614150000
1,2,Group,Group,a,notdefined,2018-06-15T17:00:00+05:00,12,3,4,0,1,-999.0,-999.0,-999.0,True,1,"[3, 6, 14, 17, 20, 22]",group_a,2018,20180615120000
2,17,Group,Group,a,notdefined,2018-06-19T21:00:00+03:00,3,1,3,3,1,-999.0,-999.0,-999.0,True,2,"[3, 6, 13, 17, 15, 21, 22]",group_a,2018,20180619180000
3,18,Group,Group,a,notdefined,2018-06-20T18:00:00+03:00,10,4,2,1,0,-999.0,-999.0,-999.0,True,2,"[3, 6, 13, 17, 21, 22]",group_a,2018,20180620150000
4,33,Group,Group,a,notdefined,2018-06-25T18:00:00+04:00,7,4,1,3,0,-999.0,-999.0,-999.0,True,3,"[4, 6, 13, 18, 15, 20, 22]",group_a,2018,20180625140000


In [175]:
df_matches_transformed.columns.to_list()

['match_id',
 'type',
 'stage',
 'group_id',
 'round_id',
 'date',
 'stadium_id',
 'home_team_id',
 'away_team_id',
 'home_result',
 'away_result',
 'home_penalty',
 'away_penalty',
 'winner',
 'finished',
 'matchday',
 'channels',
 'stage_name',
 'edition',
 'formatted_date']

### Merger les données dans un seule dataframe (teams_home , teams_away, groups , rounds , stadiums , matches)

In [176]:
#ajouter des préfixes aux colonnes des dataframes transformés:
df_teams_home= df_teams_transformed.add_prefix('team_home')
df_teams_away= df_teams_transformed.add_prefix('team_away')
df_groups_transformed = df_groups_transformed.add_prefix('group_')
df_rounds_transformed = df_rounds_transformed.add_prefix('round_')
df_stadiums_transformed = df_stadiums_transformed.add_prefix('stadium_')
# df_tvchannels_transformed = df_tvchannels_transformed.add_prefix('channel_')
df_matches_transformed = df_matches_transformed.add_prefix('match_')
df_matches_transformed
#faire le merge entre les dataframes transformés pour obtenir un dataframe final des matches complet
df_2018_final = (df_matches_transformed
                 .merge(df_teams_home, left_on='match_home_team_id', right_on='team_homeid', how='left')
                 .merge(df_teams_away, left_on='match_away_team_id', right_on='team_awayid', how='left')
                 .merge(df_groups_transformed, left_on='match_group_id', right_on='group_group_id', how='left')
                 .merge(df_rounds_transformed, left_on='match_round_id', right_on='round_round_id', how='left')
                 .merge(df_stadiums_transformed, left_on='match_stadium_id', right_on='stadium_id', how='left')
)
# df_matches_transformed.head()
list_columns_original = config['list_columns_original_2018']
list_columns_final = config['list_wanted_columns']
df_2018_final = fct_fillna_and_convert_types(df_2018_final)
df_2018_final.head()
# df_2018_final = fct_final_columns_to_keep(df_2018_final, list_columns_original, list_columns_final).sort_values(by='match_id').reset_index(drop=True)

,match_match_id,match_type,match_stage,match_group_id,match_round_id,match_date,match_stadium_id,match_home_team_id,match_away_team_id,match_home_result,...,team_awayname,group_group_id,group_group_name,group_winner_team_id,group_runnerup_team_id,round_round_id,round_round_name,stadium_id,stadium_name,stadium_city
0,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,...,Saudi arabia,a,GROUP A,4.0,1.0,notdefined,notdefined,1,Luzhniki stadium,Moscow
1,2,Group,Group,a,notdefined,2018-06-15T17:00:00+05:00,12,3,4,0,...,Uruguay,a,GROUP A,4.0,1.0,notdefined,notdefined,12,Central stadium,Yekaterinburg
2,17,Group,Group,a,notdefined,2018-06-19T21:00:00+03:00,3,1,3,3,...,Egypt,a,GROUP A,4.0,1.0,notdefined,notdefined,3,Krestovsky stadium,Saint petersburg
3,18,Group,Group,a,notdefined,2018-06-20T18:00:00+03:00,10,4,2,1,...,Saudi arabia,a,GROUP A,4.0,1.0,notdefined,notdefined,10,Rostov arena,Rostov-on-don
4,33,Group,Group,a,notdefined,2018-06-25T18:00:00+04:00,7,4,1,3,...,Russia,a,GROUP A,4.0,1.0,notdefined,notdefined,7,Cosmos arena,Samara


In [177]:
# Visualiser le DataFrame final transformé pour l'année 2018
df_2018_final.head(65)


,match_match_id,match_type,match_stage,match_group_id,match_round_id,match_date,match_stadium_id,match_home_team_id,match_away_team_id,match_home_result,...,team_awayname,group_group_id,group_group_name,group_winner_team_id,group_runnerup_team_id,round_round_id,round_round_name,stadium_id,stadium_name,stadium_city
0,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,...,Saudi arabia,a,GROUP A,4.0,1.0,notdefined,notdefined,1,Luzhniki stadium,Moscow
1,2,Group,Group,a,notdefined,2018-06-15T17:00:00+05:00,12,3,4,0,...,Uruguay,a,GROUP A,4.0,1.0,notdefined,notdefined,12,Central stadium,Yekaterinburg
2,17,Group,Group,a,notdefined,2018-06-19T21:00:00+03:00,3,1,3,3,...,Egypt,a,GROUP A,4.0,1.0,notdefined,notdefined,3,Krestovsky stadium,Saint petersburg
3,18,Group,Group,a,notdefined,2018-06-20T18:00:00+03:00,10,4,2,1,...,Saudi arabia,a,GROUP A,4.0,1.0,notdefined,notdefined,10,Rostov arena,Rostov-on-don
4,33,Group,Group,a,notdefined,2018-06-25T18:00:00+04:00,7,4,1,3,...,Russia,a,GROUP A,4.0,1.0,notdefined,notdefined,7,Cosmos arena,Samara
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,60,Winner,Knockout,notdefined,round_8,2018-07-07T18:00:00+04:00,7,23,28,0,...,England,notdefined,notdefined,-999.0,-999.0,round_8,Quarter-finals,7,Cosmos arena,Samara
60,61,Winner,Knockout,notdefined,round_4,2018-07-10T21:00:00+03:00,3,9,25,1,...,Belgium,notdefined,notdefined,-999.0,-999.0,round_4,Semi-finals,3,Krestovsky stadium,Saint petersburg
61,62,Winner,Knockout,notdefined,round_4,2018-07-11T21:00:00+03:00,1,15,28,2,...,England,notdefined,notdefined,-999.0,-999.0,round_4,Semi-finals,1,Luzhniki stadium,Moscow
62,63,Loser,Knockout,notdefined,round_2_loser,2018-07-14T17:00:00+03:00,3,25,28,2,...,England,notdefined,notdefined,-999.0,-999.0,round_2_loser,Third place play-off,3,Krestovsky stadium,Saint petersburg


### fonction finale de transformations

In [178]:
def fct_transform_data_2018(dfs_2018 : Dict[str, pd.DataFrame] , config: Dict) -> pd.DataFrame:
    """
    Transformer le DataFrame final des matches 2018 en gardant uniquement les colonnes spécifiées dans la configuration.
    
    Paramètres :
        dfs_2018 (List) : Liste des DataFrames transformés pour l'année 2018.
        config (Dict) : Dictionnaire de configuration contenant les paramètres nécessaires.
    
    Retour :
        pd.DataFrame : DataFrame final transformé pour l'année 2018.
    """
    
    df_stadiums = dfs_2018['stadiums'].copy()
    df_teams = dfs_2018['teams'].copy()
    df_groups = dfs_2018['groups'].copy()
    df_rounds = dfs_2018['rounds'].copy()
    df_matches = dfs_2018['matches'].copy()
    #-------------------------------------------------------------------------
    #------------------------stadiums transformations #------------------------
    #-------------------------------------------------------------------------

    # Filtrer les colonnes nécessaires
    columns_to_keep = ['id', 'name', 'city']
    df_stadiums_transformed = df_stadiums[columns_to_keep].copy()
    
    #traitement des valeurs nulles
    df_stadiums_transformed= fct_fillna_and_convert_types(df_stadiums_transformed)
    
    #supprimer '.' à la fin des noms des villes
    df_stadiums_transformed['city'] = df_stadiums_transformed['city'].str.replace(r'\.$', '', regex=True)

    # Mettre en le premier caractère en Majuscule pour les colonnes name et city & convertir en type string
    df_stadiums_transformed = fct_capitalize_string_columns(df_stadiums_transformed, ['name', 'city'])
    
    #-------------------------------------------------------------------------
    #------------------------teams transformations #------------------------
    #-------------------------------------------------------------------------
    # Filtrer les colonnes nécessaires
    columns_to_keep = ['id', 'name']
    df_teams_transformed = df_teams[columns_to_keep].copy()
    
    #traitement des valeurs nulles
    df_teams_transformed = fct_fillna_and_convert_types(df_teams_transformed)
    
    # Mettre en le premier caractère en Majuscule pour la colonne name & convertir en type string
    df_teams_transformed = fct_capitalize_string_columns(df_teams_transformed, ['name'])
    
    #-------------------------------------------------------------------------
    #------------------------groups transformations #------------------------
    #-------------------------------------------------------------------------
    # Mettre en le premier caractère en Majuscule pour la colonne group_name & convertir en type string
    df_groups_transformed = fct_upper_string_columns(df_groups, ['group_name'])

    # Mettre en le premier caractère en Muniscule pour la colonne group_id & convertir en type string
    df_groups_transformed = fct_lower_string_columns(df_groups_transformed, ['group_id'])
    
    #traitement des valeurs nulles
    df_groups_transformed = fct_fillna_and_convert_types(df_groups_transformed)

    #-------------------------------------------------------------------------
    #------------------------rounds transformations #------------------------
    #-------------------------------------------------------------------------
    # Mettre en le premier caractère en Minuscule pour la colonne round_id & convertir en type string
    df_rounds_transformed = fct_lower_string_columns(df_rounds, ['round_id'])

    # Mettre en le premier caractère en Majuscule pour la colonne round_name & convertir en type string
    df_rounds_transformed = fct_capitalize_string_columns(df_rounds_transformed, ['round_name'])
    
    #traitement des valeurs nulles
    df_rounds_transformed = fct_fillna_and_convert_types(df_rounds_transformed)
    
    #-------------------------------------------------------------------------
    #------------------------matches transformations #------------------------
    #-------------------------------------------------------------------------
    # Mettre en le premier caractère en Minuscule pour la colonne group_id et round_id & convertir en type string
    df_matches_transformed = fct_lower_string_columns(df_matches, ['group_id','round_id'])

    # Mettre en le premier caractère en Majuscule pour la colonne stage et type & convertir en type string
    df_matches_transformed = fct_capitalize_string_columns(df_matches_transformed, ['stage','type'])
                
    #traitement des valeurs nulles
    df_matches_transformed = fct_fillna_and_convert_types(df_matches_transformed)

    # Générer une colonne unique 'stage' en combinant les colonnes 'round_id' et 'group_id'.
    df_matches_transformed = fct_generate_unique_stage(df_matches_transformed, 'stage', 'round_id', 'group_id')

    # Extraire l'année de la colonne date pour créer la colonne edition
    df_matches_transformed['edition'] = fct_extract_edition(df_matches_transformed, 'date')['edition']

    # Convertir la colonne date ISO en format YYYYMMDDhhmmss
    df_matches_transformed = fct_iso_to_yyyymmddhhmmss(df_matches_transformed, 'date', 'formatted_date')

    # Eclater les listes dans la colonne 'channels' en plusieurs lignes
    # df_matches_transformed = df_matches_transformed.explode('channels').reset_index(drop=True)

    #Génerer une colonne unique 'stage' en combinant les colonnes 'round_id' et 'group_id'.
    df_matches_transformed = fct_generate_unique_stage(df_matches_transformed, 'stage', 'round_id', 'group_id')
    
    # Harmoniser la colonne stage_name en fonction des valeurs de stage
    stage_mapping = config['stage_mapping_2018']
    df_matches_transformed = fct_harmonize_column_values(df_matches_transformed, 'stage_name', stage_mapping)
    #-------------------------------------------------------------------------
    #-----------------------Merger en dataframe finale------------------------
    #-------------------------------------------------------------------------
    #ajouter des préfixes aux colonnes des dataframes transformés:
    df_teams_home= df_teams_transformed.add_prefix('team_home')
    df_teams_away= df_teams_transformed.add_prefix('team_away')
    df_groups_transformed = df_groups_transformed.add_prefix('group_')
    df_rounds_transformed = df_rounds_transformed.add_prefix('round_')
    df_stadiums_transformed = df_stadiums_transformed.add_prefix('stadium_')
    # df_tvchannels_transformed = df_tvchannels_transformed.add_prefix('channel_')
    df_matches_transformed = df_matches_transformed.add_prefix('match_')

    #faire le merge entre les dataframes transformés pour obtenir un dataframe final des matches complet
    df_2018_final = (df_matches_transformed
                    .merge(df_teams_home, left_on='match_home_team_id', right_on='team_homeid', how='left')
                    .merge(df_teams_away, left_on='match_away_team_id', right_on='team_awayid', how='left')
                    .merge(df_groups_transformed, left_on='match_group_id', right_on='group_group_id', how='left')
                    .merge(df_rounds_transformed, left_on='match_round_id', right_on='round_round_id', how='left')
                    .merge(df_stadiums_transformed, left_on='match_stadium_id', right_on='stadium_id', how='left')
    )
    
     #traitement des valeurs nulles après les merges
    df_2018_final= fct_fillna_and_convert_types(df_2018_final)
                
    #-------------------------------------------------------------------------
    #--------------------Garder que les colonnes demandées--------------------
    #-------------------------------------------------------------------------
    list_columns_original = config['list_columns_original_2018']
    list_columns_final = config['list_wanted_columns']
    df_2018_final = fct_final_columns_to_keep(df_2018_final, list_columns_original, list_columns_final).sort_values(by='match_id').reset_index(drop=True)
    
    return df_2018_final
        
